**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 Absent.gdoc
'Academic Transcript.pdf'
 Adh-chaya.gdoc
 approval_page.pdf
'Colab Notebooks'
'Cuisine’s i have tried yet in us.gdoc'
 CV.pdf
 Dc.gdoc
 DSC00788.JPG
 DSC00792.JPG
 DSC00823.JPG
 DSC00843.JPG
 DSC00846.JPG
 DSC00847.JPG
 EmiratesOnlineCheckin.pdf
'FA for coursera.gdoc'
 Gifts.gdoc
'Grocery for Diwali.gdoc'
'HL7 Lab Results HL7 Lab Results_04162021_2021050411520537226.pdf'
 IMG_0643.mp4
 IMG_2150.JPG
 Important.gdoc
'Indian bajar.gdoc'
 K-state_problem.gdoc
'List of food have to eat bari gie.gdoc'
'List of restaurants to visit.gdoc'
 Makeup.gdoc
'Math 108-006 .gsheet'
'Meal Prep.gdoc'
'Montreal, Canada.gdoc'
 movies
'My Drive'
 Proposal.pdf
'Ramkrishna boleche.gdoc'
 reception_videos
 RESUME1.pdf
 Resume__RituparnaBasak-converted.docx
 resume+Rituparna+Basak.pdf
'RituparnaBasak-Resume for Data Scientist.pdf'
 Sari.gdoc
'Sepsis_logistic_regreesion-checkpoint (1).ipynb'
 Sepsis_logistic_regreesion-checkpoint.ipynb
 Shoprite.gdoc
 SOP_UMBC.pdf
'State

**1) Clone the Darknet**



In [ ]:
!rm -r darknet
!git clone https://github.com/AlexeyAB/darknet

rm: cannot remove 'darknet': No such file or directory
Cloning into 'darknet'...
remote: Enumerating objects: 15825, done.
remote: Counting objects: 100% (286/286), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 15825 (delta 148), reused 202 (delta 121), pack-reused 15539
Receiving objects: 100% (15825/15825), 14.76 MiB | 16.78 MiB/s, done.
Resolving deltas: 100% (10578/10578), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: un

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training.cfg
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Toy\n\nPig1\nPig2" > data/obj.names
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!rm -r data/obj
!mkdir data/obj

rm: cannot remove 'data/obj': No such file or directory


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-10-03 20:59:08--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  60.1MB/s    in 2.6s    

2023-10-03 20:59:11 (60.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
   creating: data/obj/images/
  inflating: data/obj/__MACOSX/._images  
  inflating: data/obj/images/sample_video_img_1419.txt  
  inflating: data/obj/__MACOSX/images/._sample_video_img_1419.txt  
  inflating: data/obj/images/sample_video_img_0920.txt  
  inflating: data/obj/__MACOSX/images/._sample_video_img_0920.txt  
  inflating: data/obj/images/sample_video_img_8603.txt  
  inflating: data/obj/__MACOSX/images/._sample_video_img_8603.txt  
  inflating: data/obj/images/sample_video_img_4978.txt  
  inflating: data/obj/__MACOSX/images/._sample_video_img_4978.txt  
  inflating: data/obj/images/sample_video_img_5482.txt  
  inflating: data/obj/__MACOSX/images/._sample_video_img_5482.txt  
  inflating: data/obj/images/sample_video_img_6799.txt  
  inflating: data/obj/__MACOSX/images/._sample_video_img_6799.txt  
  inflating: data/obj/images/sample_video_img_5345.jpg  
  inflating: data/obj/__MACOSX/images/._sample_video_img_5345.jpg  
  inflating: dat

In [ ]:
# We're going to convert the class index on the .txt files. yolov3 annotation formating
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        if('classes' in file_path):
            continue

        text_converted = []
        for line in lines:
            #print("\nthis is the line: ", line)
            numbers = re.findall("[0-9.]+", line)
            #print("this is the numbers: ", numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                #print(i, file_path)
                #print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)


this is the line:  1 0.234115 0.290278 0.059896 0.215741

this is the numbers:  ['1', '0.234115', '0.290278', '0.059896', '0.215741']
0 data/obj/images/sample_video_img_4938.txt
1 0.234115 0.290278 0.059896 0.215741

this is the line:  2 0.283594 0.685185 0.071354 0.105556

this is the numbers:  ['2', '0.283594', '0.685185', '0.071354', '0.105556']
0 data/obj/images/sample_video_img_4938.txt
2 0.283594 0.685185 0.071354 0.105556

this is the line:  0 0.403906 0.794444 0.089063 0.161111

this is the numbers:  ['0', '0.403906', '0.794444', '0.089063', '0.161111']
0 data/obj/images/sample_video_img_4938.txt
0 0.403906 0.794444 0.089063 0.161111

this is the line:  1 0.372656 0.701389 0.188021 0.347222

this is the numbers:  ['1', '0.372656', '0.701389', '0.188021', '0.347222']
1 data/obj/images/sample_video_img_1419.txt
1 0.372656 0.701389 0.188021 0.347222

this is the line:  2 0.499740 0.676389 0.124479 0.299074

this is the numbers:  ['2', '0.499740', '0.676389', '0.124479', '0.299074

In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(len(images_list))

132


In [ ]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.808840), count: 4, class_loss = 0.003211, iou_loss = 0.123807, total_loss = 0.127018 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.003436, iou_loss = 0.000000, total_loss = 0.003436 
 total_bbox = 322017, rewritten_bbox = 0.031675 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000008, iou_loss = 0.000000, total_loss = 0.000008 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.854907), count: 9, class_loss = 0.168959, iou_loss = 0.181957, total_loss = 0.350916 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 322026, rewritten_bbox = 0.031674 % 
v3 (m